## Harbor Installation
Dieses Notebook enthält die notwendigen Befehle zur Installation und Konfiguration von Harbor mittels Docker Compose.


### Harbor Installation

In [ ]:
%%bash
sudo apt-get install docker-compose -y

In [ ]:
%%bash
wget https://github.com/goharbor/harbor/releases/download/v2.12.2/harbor-offline-installer-v2.12.2.tgz
tar xzf harbor-offline-installer-v2.12.2.tgz
cd harbor

### TLS Zertifikate vom Cert-Manager abrufen

In [ ]:
%%bash
cd harbor
kubectl get secret selfsigned-cert-secret -n m01-ch -o jsonpath="{.data.tls\.crt}" | base64 --decode > tls.crt
kubectl get secret selfsigned-cert-secret -n m01-ch -o jsonpath="{.data.tls\.key}" | base64 --decode > tls.key

### Zertifikatsinhalt prüfen

In [ ]:
%%bash
cd harbor
openssl x509 -in tls.crt -text -noout

### Harbor Konfiguration anpassen

In [ ]:
%%bash
cd harbor
cp harbor.yml.tmpl harbor.yml
sed -i -e "s/reg.mydomain.com/ch.$(cat ~/work/server-ip)-edutbz.com/g" harbor.yml
sed -i -e 's/port: 80/port: 9090/g' harbor.yml
sed -i -e 's/port: 443/port: 9443/g' harbor.yml
sed -i -e 's;certificate: /your/certificate/path;certificate: /home/ubuntu/cascpm/A-infra/harbor/tls.crt;g' harbor.yml
sed -i -e 's;private_key: /your/private/key/path;private_key: /home/ubuntu/cascpm/A-infra/harbor/tls.key;g' harbor.yml

Folgende Änderungen in `harbor.yml` vornehmen:

- `hostname: reg.mydomain.com` → `ch.cloud-hf-08-edutbz.com`

- `http.port: 80` → `8080`

- `https.port: 443` → `9443`

- `certificate: /your/certificate/path` → `/home/ubuntu/helm/tls.crt`

- `private_key: /your/private/key/path` → `/home/ubuntu/helm/tls.key`


### Harbor starten

In [ ]:
%%bash
cd harbor
sudo ./install.sh --with-trivy

Harbor ist dann via dem untenstehenden URL erreichbar.
Benutzer: `admin`, Passwort: `Harbor12345`.

In [ ]:
%%bash
echo "https://ch.$(cat ~/work/server-ip)-edutbz.com:9443"

### Docker: Selbst signiertes Zertifikat vertrauenswürdig machen

In [ ]:
%%bash
cd harbor
kubectl get secret selfsigned-cert-secret -n m01-ch -o jsonpath="{.data.ca\.crt}" | base64 --decode > ca.crt

In [ ]:
%%bash
cd harbor
sudo mkdir -p /etc/docker/certs.d/ch.$(cat ~/work/server-ip)-edutbz.com:9443
sudo cp ca.crt /etc/docker/certs.d/ch.$(cat ~/work/server-ip)-edutbz.com:9443/ca.crt

Docker und Harbor neu starten, damit das Zertifikat geladen wird:

In [ ]:
%%bash
cd harbor
sudo docker-compose down -v
sudo systemctl restart docker
sudo ./install.sh --with-trivy

Jetzt sollte `docker login` ohne Fehlermeldung funktionieren.

    docker login ch.cloud-hf-08-edutbz.com:9443

### microk8s: Selbst signiertes Zertifikat vertrauenswürdig machen (nicht getestet)

In [ ]:
%%bash
cd harbor
sudo mkdir -p /var/snap/microk8s/current/certs.d/ch.$(cat ~/work/server-ip)-edutbz.com:9443
sudo cp ca.crt /var/snap/microk8s/current/certs.d/ch.$(cat ~/work/server-ip)-edutbz.com:9443/ca.crt

sudo microk8s stop
sudo microk8s start

### Aufträge
- Repliziert Eure Container Registries bzw. die Images nach Harbor.
- Erstellt ein Container Image und legt es in die Harbor Registry ab.

**Links:**
[Harbor Installer](https://goharbor.io/docs/2.12.0/install-config/run-installer-script/)